In [ ]:
pip install flask

In [ ]:
!pip install flask-cors
!pip install flask-ngrok


In [ ]:
!python --version


Python 3.7.12


In [ ]:
from flask import Flask
from keras.models import load_model
import cv2
from keras.models import model_from_json
from flask_ngrok import run_with_ngrok
import numpy as np
from flask_cors import CORS
from flask import jsonify
import pandas as pd
# from __future__ import unicode_literals
import json
import time
from flask import request
from keras.preprocessing.image import load_img, img_to_array

from copy import deepcopy



BASE_PATH = "/content/drive/MyDrive/HealthAppProject/"

MODEL_PATH = "/content/drive/MyDrive/content/model.h5"
app = Flask(__name__)
run_with_ngrok(app)


# LOAD THE MODEL HERE
json_file = open(f'{BASE_PATH}model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
print(f"{BASE_PATH}model.h5")

#loaded_model.load_weights(f"{BASE_PATH}model.h5")

loaded_model.load_weights(MODEL_PATH)

data_path = f'{BASE_PATH}nutri.csv'



@app.route("/foodchecker")
def home():
    return "Welcome to flask backend !"


Name = ['mysore_pak', 'unni_appam', 'paneer_butter_masala', 'jalebi', 'biryani', 'chapati']

N = []
for i in range(len(Name)):
    N += [i]

mapping = dict(zip(Name, N))
reverse_mapping = dict(zip(N, Name))


def mapper(value):
    return reverse_mapping[value]

calorie_data = pd.read_csv(data_path)
calorie_data.head()



def get_formatted_response(unformatted_dict):

  copy_unformatted_dict = {}

  for key, value in unformatted_dict.items():

    if str(value) == 'nan':
      value = False
    if "percentage" in key.lower():
      continue
    elif "food name" in key.lower():
      copy_unformatted_dict["FOOD_NAME"] = value
    elif "gram" in key.lower():
      copy_unformatted_dict[key.split("_")[0].upper()] = value
    else:
      copy_unformatted_dict[key.upper()] = value

    print()

  return deepcopy(copy_unformatted_dict)






def load_calorie_data(name_of_food):
    calorie_data_of_food = calorie_data[calorie_data['Food Name'] == name_of_food]
    print('------------------------------------------------')
  
    #print(calorie_data_of_food.to_dict(orient='records'))


    return calorie_data_of_food


def upadteEffects(calorie_data_of_food, expectedMacroNutrients,userData):

  if(userData['hasDaibeties'] and  calorie_data_of_food['category'] == 'sugar'):
    calorie_data_of_food['effect_on_sugar'] = True;
  
  if(userData['hasHighCholestrol'] and  'fat' ):
    expec_per = float(expectedMacroNutrients['FATS']['PERCENTAGE'])
    fats_per = float(calorie_data_of_food['fats_percentage'])


    calorie_data_of_food['effect_on_chol'] = fats_per > expec_per;
    
  return calorie_data_of_food


@app.route("/",  methods = ['GET', 'POST'])
def predict_by_model():
    try:
        img_data = request.args.get('imageUri')
        image_64_decode = base64.decodestring(img_data) 
        image_result = open('food.jpg', 'wb') # writeable image 
        image_result.write(image_64_decode)
        image = load_img("food.jpg", target_size=(40, 40))
        
        #user_data = request.args.get('userData')
        userData = {
          "age":53,
          "bodySugar":111,
          "diastolicBloodPressure":55,
          "gender":"Male",
          "hasDaibeties":false,
          "hasHighCholestrol":false,
          "hasHypertension":false,
          "height":153,
          "pregnancyStatus":false,
          "systolicBloodPressure":120,
          "weight":88
          }

      expectedMacroNutrients = {
          "CALORIES":{
            "gram":2167.34375
          },
          "CARBS":{
            "gram":298,
            "percentage":55
          },
          "FATS":{
            "gram":67,
            "percentage":28
          },
          "PROTIEN":{
            "gram":125,
            "percentage":23
          }
      }

        # preprocessing the image
        image = img_to_array(image)
        image = image / 255.0
        prediction_image = np.array(image)
        prediction_image = np.expand_dims(image, axis=0)
        # predict the model
        prediction = loaded_model.predict(prediction_image)
        value = np.argmax(prediction)
        food_name = mapper(value)
        print("Prediction is {}.".format(food_name))

        calorie_data_of_food = load_calorie_data(food_name)

        calorie_data_of_food = upadteEffects(upadteEffects,expectedMacroNutrients,userData)

        calorie_data_of_food = calorie_data_of_food.to_dict(orient='records')

        '''dictionary_response = {'name_of_food': food_name, 'CALORIES': calorie_data_of_food['calories'],
                               'CARBS': calorie_data_of_food['carbs_gram'], 'FATS': calorie_data_of_food['fats_gram'],
                               'PROTIEN': calorie_data_of_food['protien_gram'], 'SUITABLE_DIABETES': '',
                               'SUITABLE_CHOLESTROL': '', 'SUITABLE_PREGNANCY': ''}

        print(dictionary_response)

        json_response = json.dumps(dictionary_response)'''

        '''
      "userData":{
      "age":53,
      "bodySugar":111,
      "diastolicBloodPressure":55,
      "gender":"Male",
      "hasDaibeties":false,
      "hasHighCholestrol":false,
      "hasHypertension":false,
      "height":153,
      "pregnancyStatus":false,
      "systolicBloodPressure":120,
      "weight":88 }
        '''

        formatted_response = get_formatted_response(calorie_data_of_food[0])        
        print(formatted_response)

        return json.dumps(formatted_response)

    except Exception as e:
        return jsonify(error=str(e))

#/content/model.h5

CORS(app)
app.run()

/content/drive/MyDrive/HealthAppProject/model.h5
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://50c7-34-86-219-56.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Nov/2021 16:45:16] "GET / HTTP/1.1" 200 -


Prediction is biryani.
------------------------------------------------










{'FOOD_NAME': 'biryani', 'CARBS': 56, 'FATS': 161, 'PROTIEN': 19, 'CALORIES': 241, 'EFFECT_ON_SUGAR ': False, 'EFFECT_ON_CHOL': False, 'EFFECT_ON_PREGNANCY': False, 'EFFECT_ON_BP': False, 'SERVING_SIZE': False}


...

Prediction is biryani.
------------------------------------------------










{'FOOD_NAME': 'biryani', 'CARBS': 56, 'FATS': 161, 'PROTIEN': 19, 'CALORIES': 241, 'EFFECT_ON_SUGAR ': False, 'EFFECT_ON_CHOL': False, 'EFFECT_ON_PREGNANCY': False, 'EFFECT_ON_BP': False, 'SERVING_SIZE': False}


127.0.0.1 - - [14/Nov/2021 16:47:21] "POST / HTTP/1.1" 200 -


Prediction is biryani.
------------------------------------------------










{'FOOD_NAME': 'biryani', 'CARBS': 56, 'FATS': 161, 'PROTIEN': 19, 'CALORIES': 241, 'EFFECT_ON_SUGAR ': False, 'EFFECT_ON_CHOL': False, 'EFFECT_ON_PREGNANCY': False, 'EFFECT_ON_BP': False, 'SERVING_SIZE': False}


127.0.0.1 - - [14/Nov/2021 16:47:29] "POST / HTTP/1.1" 200 -


Prediction is biryani.
------------------------------------------------










{'FOOD_NAME': 'biryani', 'CARBS': 56, 'FATS': 161, 'PROTIEN': 19, 'CALORIES': 241, 'EFFECT_ON_SUGAR ': False, 'EFFECT_ON_CHOL': False, 'EFFECT_ON_PREGNANCY': False, 'EFFECT_ON_BP': False, 'SERVING_SIZE': False}


In [ ]:
import base64

file = 'deer.jpg'
image = open(file, 'rb')
image_read = image.read()
image_64_encode = base64.encodebytes(image_read) #encodestring also works aswell as decodestring


image_64_decode = base64.decodestring(image_64_encode) 
image_result = open('deer_decode.gif', 'wb') # create a writable image and write the decoding result
image_result.write(image_64_decode)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip freeze > requirements.txt


In [ ]:
pip install gunicorn


     |████████████████████████████████| 79 kB 2.9 MB/s 


In [ ]:
pip freeze > requirements.txt